### General analysis of november data

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
from retrieve_data import read_waze_data
import os

In [3]:
[df_tt, df_r] = read_waze_data('21.11.2018')

In [4]:
#df_tt.head()

In [5]:
def process_waze_data(df_tt, df_r):

    df_tt['date'] = df_tt['updatetime'].dt.date
    df_tt['time_ext'] = df_tt['updatetime'].dt.time

    df_tt = df_tt.merge(df_r[['name', 'length']], on='name', how='left')
    df_tt = df_tt.loc[df_tt['length'].isnull() == False, :]
    df_tt['time/length[s/km]'] = (df_tt['time'] / (df_tt['length']))*1000
    df_tt['length/time[km/h]'] = (df_tt['length'] / (df_tt['time']))*3.6

    return df_tt

In [6]:
df_tt = process_waze_data(df_tt, df_r)

In [7]:
#df_tt.head()

Select the dates

In [8]:
import datetime as dt

In [9]:
start_date = dt.date(2018, 11, 5)
except_7_nov = dt.date(2018,11,7) #energy cut-off
except_12_nov = dt.date(2018,11,12) #hail storm
end_date = dt.date(2018, 11, 18)

In [10]:
mask = (df_tt['date'] >= start_date) & (df_tt['date'] <= end_date) & (df_tt['date'] != except_7_nov) & (df_tt['date'] != except_12_nov)
nov_df = df_tt.loc[mask]
print(len(nov_df.index))

200604


In [11]:
include_24_oct = dt.date(2018,10,24)
include_19_nov = dt.date(2018,11,19)

In [12]:
include_mask = (df_tt['date'] == include_24_oct) | (df_tt['date'] == include_19_nov)
replacing_df = df_tt.loc[include_mask]
print(len(replacing_df.index))

31392


In [13]:
analyze_df = pd.concat([nov_df,replacing_df])
print(len(analyze_df.index))

231996


In [14]:
def day_type(x):
    if (0<=x.weekday()<5):
        return 'L'
    elif (x.weekday()==5):
        return 'S'
    elif (x.weekday()==6):
        return 'D'

In [15]:
analyze_df['day'] = analyze_df['date'].apply(day_type)

In [16]:
#analyze_df.head()

In [17]:
grouped_df_tt = analyze_df.groupby(['name', 'day', pd.Grouper(freq='15min', key='updatetime')])['time/length[s/km]','length/time[km/h]'].mean()
grouped_df_tt.reset_index(inplace=True)

In [18]:
#grouped_df_tt.head()